In [ ]:
import pandas as pd
import processor as pr
import loader as load
import predictor as pred
import config

In [ ]:

data, files = load.loadAll(includeStage=("stage"), sameSamples=True, skipGenes=True)
ae, ae_file = data[-1], files[-1]
print(len(ae))
ae = load.attachStageStatus(ae)
print(len(ae))
x, y = pr.splitData(ae, target="stage", project="COAD")

In [ ]:
y

In [ ]:
stad_stage_exp = True
# aak_ge takes a while. chokes during feature selection COAD even with 5
for target in config.prediction_targets[1:]:
    data, files = load.loadAll(includeStage=(target=="stage"), sameSamples=True, skipGenes=False)
    for sampling in config.sampling[:]:
        for selection in config.selection_types[:2]:
            # pred.runExperiments(data[1:2], files[1:2], target=target, sampling=sampling, selection=selection)
            
            for parity in config.modality_parities[:1]:
                enforce_modality_parity = (parity == "parity")

                # Only run for overlap set if you run with parity
                if enforce_modality_parity:
                    genus_overlapping_ge_name = "tcma_gen_aak_ge"
                    genus_overlapping_ge  = load.getSpecificData(genus_overlapping_ge_name, data, files)
                    pred.runExperiments([genus_overlapping_ge], [genus_overlapping_ge_name], target=target, sampling=sampling, selection=selection, modality_selection_parity=enforce_modality_parity, stad_exp=stad_stage_exp)
                else:
                    pred.runExperiments(data[:], files[:], target=target, sampling=sampling, selection=selection, modality_selection_parity=enforce_modality_parity, stad_exp=stad_stage_exp)

In [ ]:
data, files = load.loadAll(includeStage=True, sameSamples=True)

In [ ]:
pred.runExperiments(data[:], files[:], target="stage", sampling="random_sampling")

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, average_precision_score, log_loss
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from tuner import getTunedModel

d = data[2]
d = load.attachStageStatus(d)
x, y = pr.splitData(d, target="stage", project="STAD")
# print(y)
model = LinearRegression()
# pred.runRandomSampling(x, y, model)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)

model.fit(x_train, y_train)
y_predicted = model.predict(x_test)
y_predicted_rounded = np.rint(y_predicted)
# y_predicted_rounded
# y_predicted
# y.dtype
print(model.coef_)


def printImportances(name, features, coefficients):
    with open(name, "w+") as f:
        f.write(str(list(zip(features, coefficients))))
# printImportances("z_linreg_coef", x_train.columns, model.coef_.tolist())

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso
# model = ElasticNet(random_state=0)
# model = Lasso(random_state=0)
model = RandomForestRegressor(random_state=0)

model.fit(x_train, y_train)
# tuned_object = getTunedModel(model, x_train, y_train, random_state=42, scoring="neg_root_mean_squared_error")

In [ ]:
# [tree.feature_importances_.tolist() for tree in tuned_object.best_estimator_.estimators_],
from sklearn.inspection import permutation_importance
forest = tuned_object.best_estimator_
# result = permutation_importance(
#     forest, x_test, y_test, n_repeats=10, random_state=42, n_jobs=-1
# )


In [ ]:
# result.importances_mean.tolist()

In [ ]:

# model = tuned_object.best_estimator_
# best_parameters = tuned_object.best_params_
# print(f"Finished tuning param: {best_parameters}")
model.feature_importances_
method = "rfreg"

In [ ]:
coefficients = model.feature_importances_ if method == "rfreg" else model.coef_
print(coefficients.tolist())
# features_with_coefficients = pd.DataFrame(zip(x_train.columns, coefficients))
features_with_coefficients = pd.DataFrame({"feature":x_train.columns,"coefficients":np.transpose(coefficients)})
print(features_with_coefficients)
features_with_coefficients_abs = features_with_coefficients.copy()
features_with_coefficients_abs["coefficients"] = features_with_coefficients_abs.apply(lambda row: abs(row.coefficients), axis=1)
features_with_coefficients_abs
sorted_features = features_with_coefficients_abs.sort_values("coefficients", ascending=False)
top_features = sorted_features.head(20)["feature"].values
x_train[top_features]
# printImportances("z_rf_tuned_coef", x_train.columns, coefficients.tolist())
print(top_features)
# print(coefficients)
best_indices = [x_train.columns.get_loc(c) for c in top_features]


print(x_train[top_features].columns)
print(sorted_features.set_index("feature").T.to_dict("records")[0])
# print(x_train.iloc[:, best_indices].columns)
# linreg_ranked_features = pr.selectFeatures(x, y, 20, "linreg")
# print(x_train.iloc[:, linreg_ranked_features].columns)
# linreg_ranked_features[:20]

In [ ]:
# Experiments with separate feature selection per modality
all_features, _ = load.getFeatures()
gen, ge = all_features

# x_train[gen[:2]]
# files
x_train_with_genus_features = x_train[gen]
x_train_with_ge_features = x_train[ge]
# x_train_with_genus_features
x_train_with_ge_features